In [ ]:
import pandas as pd

from src.data_processing.wrangling import cat_conditions, map_comp, map_investimento, map_importo

In [ ]:
p01 = pd.read_parquet("data/updated/p01.parquet")
p02 = pd.read_parquet("data/updated/p02.parquet")
p03 = pd.read_parquet("data/updated/p03.parquet")
p04 = pd.read_parquet("data/updated/p04.parquet")
p05 = pd.read_parquet("data/updated/p05.parquet")

componenti = pd.read_parquet("data/updated/components.parquet")
missions = pd.read_parquet("data/updated/missions.parquet")
comuni = pd.read_parquet("data/updated/comuni.parquet")

## Data preprocessing

In [ ]:
p01.shape

In [ ]:
p01.columns

In [ ]:
p02.columns

In [ ]:
p02.shape

In [ ]:
p02.head()

In [ ]:
p03 = p03.rename(columns={'cig':'CIG'})
p03['CATEGORIA'] = p03.apply(cat_conditions,axis=1)

In [ ]:
print(p03.shape)
print(p03.columns)
p03.head()

In [ ]:
p04 = p04.rename(columns={'cig':'CIG'})
print(p04.shape)
print(p04.columns)
p04.head()

In [ ]:
p05['importo_complessivo_gara'].unique()

In [ ]:
p05 = p05.rename(columns={'cig':'CIG'})
p05['importo_complessivo_gara'] = p05['importo_complessivo_gara'].astype(float)
p05['CLASSE_IMPORTO'] = p05['importo_complessivo_gara'].apply(map_importo)

In [ ]:
print(p05.shape)
print(p05.columns)
p05.head()

## Merge dei Dataset "P0"

In [ ]:
# missioni e componenti, left join da p02
p02 = p02.merge(missions,left_on="Missione", right_on=missions.index, how='inner')
p02 = p02.drop_duplicates()
p02 = p02.merge(componenti,left_on="Componente", right_on=componenti.index, how='inner')
p02 = p02.drop_duplicates()

In [ ]:
comuni.columns

In [ ]:
p05.columns

In [ ]:
p05['luogo_istat']

In [ ]:
comuni['Codice Comune Alfanumerico'] = comuni['Codice Comune Alfanumerico'].astype(str)
comuni['Codice Comune Alfanumerico']

In [ ]:
# left join p05 con i dati su comuni e denominazioni 
p05=p05.merge(
    comuni,
    left_on='luogo_istat', 
    right_on='Codice Comune Alfanumerico',
    how='left'
)
p05=p05.drop_duplicates()

In [ ]:
# left join p5 e p3
leftnew_3 = p05.merge(p03, on='CIG', how='left')
leftnew_3 = leftnew_3.drop_duplicates()
print(leftnew_3.shape)

In [ ]:
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
print(leftnew_34.shape)

In [ ]:
# left join il risultato con p1
leftnew_341 = leftnew_34.merge(p01, on='CIG', how='left')
leftnew_341 = leftnew_341.drop_duplicates()
print(leftnew_341.shape)

In [ ]:
# left join il risultato con p2
leftnew_3412 = leftnew_341.merge(p02,how='left',left_on='CUP',right_on='CUP')
leftnew_3412 = leftnew_3412.drop_duplicates()
print(leftnew_3412.shape)

## Colonne da selezionare per Master Table

In [ ]:
p02.columns

In [ ]:
colonne_da_selezionare = [
    'CIG', 
    'CUP',  
    'Regione', 
    'provincia',  
    'Comune', 
    'importo_complessivo_gara', 
    'CLASSE_IMPORTO', 
    'Missione', 
    'Descrizione Missione',
    'Componente',  
    'Descrizione Componente',
    'ID Misura', 
    'Codice Univoco Misura',
    'Descrizione Misura', 
    'ID Submisura', 
    'Codice Univoco Submisura', 
    'Descrizione Submisura',
    'cod_mis_premiale', 
    'misura_premiale', 
    'CATEGORIA', 
    'flag_quote', 
    'quota_femminile', 
    'quota_giovanile', 
    'cod_mot_deroga', 
    'mot_deroga', 
    'data_pubblicazione', 
    'anno_pubblicazione', 
    'FLAG_URGENZA', 
    'MOTIVO_URGENZA', 
    'ESITO',
]

In [ ]:
master_table = leftnew_3412[colonne_da_selezionare]

In [ ]:
master_table['Comune'].nunique()

In [ ]:
master_table.shape

In [ ]:
master_table.head(5)

In [ ]:
master_table['Regione'].isna().sum()

In [ ]:
master_table['Regione'].unique()

In [ ]:
master_table.to_csv("data/updated/master_table_base.csv", sep=";")